# This notebook gets the results from the DB to process into histograms and others

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [6]:
def connection(user,passwd,dbname, echo_i=False):
    str1 = ('postgresql+pg8000://' + user +':' + passw + '@switch-db2.erg.berkeley.edu:5433/' 
            + dbname + '?ssl=true&sslfactory=org.postgresql.ssl.NonValidatingFactory')
    engine = create_engine(str1,echo=echo_i)
    return engine

In [7]:
user = 'jdlara'
passw = ''
dbname = 'apl_cec' 
engine= connection(user,passw,dbname)

In [14]:
query = ('set search_path = lemmav2, public;' +
        'with d_bm_stats as ('+
            'select min("D_BM_kg")/1000 as min,' +
                   'max("D_BM_kg")/1000 as max' +
              'from lemmav2.lemma_total where "D_BM_kg" > 100)' +
            'select width_bucket(("D_BM_kg")/1000, min, max, 40) as bucket,' +
                    'int4range(min(("D_BM_kg")/1000)::INT, max(("D_BM_kg")/1000)::INT, \'[]\') as range,' +
                    'count(*) as freq' +
                'from lemmav2.lemma_total, d_bm_stats where "D_BM_kg" > 100' +
            'group by bucket' +
            'order by bucket;')
query

'set search_path = lemmav2, public;with d_bm_stats as (select min("D_BM_kg")/1000 as min,max("D_BM_kg")/1000 as maxfrom lemmav2.lemma_total where "D_BM_kg" > 100)select width_bucket(("D_BM_kg")/1000, min, max, 40) as bucket,int4range(min(("D_BM_kg")/1000)::INT, max(("D_BM_kg")/1000)::INT, \'[]\') as range,count(*) as freqfrom lemmav2.lemma_total, d_bm_stats where "D_BM_kg" > 100group by bucketorder by bucket;'

In [8]:
result = pd.read_sql_query(query,engine)

,count
0,27949
